In [1]:
#!/usr/bin/env python
# coding: utf-8
import re
import sys
import pandas as pd
import sys
import ast
import matplotlib.pyplot as plt
import os
import time
import ast


global _app_name
run_command="/usr/lib/jvm/java-17-openjdk-amd64/bin/java -Dfile.encoding=UTF-8 -Dstdout.encoding=UTF-8 -Dstderr.encoding=UTF-8 -classpath /media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/_output/production/iFogSim2:/media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/jars/cloudsim-3.0.3.jar:/media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/jars/cloudsim-3.0.3-sources.jar:/media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/jars/cloudsim-examples-3.0.3.jar:/media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/jars/cloudsim-examples-3.0.3-sources.jar:/media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/jars/commons-math3-3.5/commons-math3-3.5.jar:/media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/jars/guava-18.0.jar:/media/saeedeh/Data/PhD/Simulators/iFog_Simulator/IFogSim/jars/json-simple-1.1.1.jar -XX:+ShowCodeDetailsInExceptionMessages org.fog.test.perfeval."

_app_name="LabellingDCNS"
_report_path="Report_Saeedeh/label_pandas.txt"
_save_path="Report_Saeedeh/Labelling_Results"


In [2]:
def run(app_name="LabellingDCNS", scenario="Router_Proxy", report_path=_report_path): 
    #cmd=f'{run_command}{app_name} {scenario} | tee -a {report_path} >/dev/null'
    cmd=f'{run_command}{app_name} {scenario} 2>&1 >> {report_path}'
    os.system(cmd)

In [3]:
# init
global Scenario,data_frames
Scenario=None
data_frames=None



# Initialize and maintain other DataFrames as global variables

if 'computation_energy_df' not in globals():
    computation_energy_df = pd.DataFrame(columns=['scenario','device', 'cmp_energy'])
    
if 'computation_co2_df' not in globals():
    computation_co2_df = pd.DataFrame(columns=['scenario','device', 'cmp_emission'])
    
if 'networking_energy_df' not in globals():
    networking_energy_df = pd.DataFrame(columns=['scenario','device', 'nw_energy'])

if 'networking_co2_df' not in globals():
    networking_co2_df = pd.DataFrame(columns=['scenario','device', 'nw_emission'])

#if 'execution_time_df' not in globals():
#    execution_time_df = pd.DataFrame(columns=['app_class','scenario','tuple', 'execution_time'])
if 'loop_delay_df' not in globals():
    loop_delay_df = pd.DataFrame(columns=['scenario','loop_name', 'delay'])
    
    
if 'simulation_time_df' not in globals():
    simulation_time_df = pd.DataFrame(columns=['scenario', 'simulation_time'])

In [4]:
# reset: remove report and clear dfs
def _reset():
    #os.system(f'rm {_report_path}')
    
    computation_energy_df = pd.DataFrame(columns=['scenario','device', 'cmp_energy'])
    networking_energy_df = pd.DataFrame(columns=['scenario','device', 'nw_energy'])
    computation_co2_df = pd.DataFrame(columns=['scenario','device', 'cmp_emission'])
    networking_co2_df = pd.DataFrame(columns=['scenario','device', 'nw_emission'])
    #execution_time_df = pd.DataFrame(columns=['app_class','scenario','tuple', 'execution_time'])
    loop_delay_df = pd.DataFrame(columns=['scenario','loop_name', 'delay'])
    simulation_time_df = pd.DataFrame(columns=['scenario', 'simulation_time'])
    time.sleep(3)

In [5]:
# Parsing Functions
# string map to map
def parse_map_string(sample):
    # Use regex to convert the string into a valid JSON format
    formatted_sample = re.sub(r'(\w+)=', r'"\1":', sample)
    formatted_sample = re.sub(r'(\w+)(?==)', r'"\1"', formatted_sample)

    # Parse the formatted string into a dictionary
    parsed_sample = ast.literal_eval(formatted_sample)

    #print(parsed_sample)
    return parsed_sample



def parse_computation_energy(l):
    global computation_energy_df
    
    # Pattern to match lines like "cloud : Application dcns total Energy = 366019.328579134"
    # or "proxy-server : Application total Energy = 0.0"
    pattern = r'^(.*?): Application(?: (.*?))? total Energy = (\d+(\.\d+)?(E[+-]?\d+)?)$'
    
    match = re.match(pattern, l)
    
    if match:
        device = match.group(1)  # Device name (e.g., cloud, proxy-server)
        app = match.group(2) if match.group(2) else 'no_app'  # Application name (e.g., dcns) or 'no_app' if missing
        energy = float(match.group(3))  # Energy value, converting to float (supports scientific notation)
        
        # Add parsed data to DataFrame
        data = {'scenario': Scenario, 'device': device, 'app': app, 'cmp_energy': energy}
        df = pd.DataFrame([data])
        
        # Combine the result with the existing DataFrame
        computation_energy_df = pd.concat([computation_energy_df, df], ignore_index=True)

        
def parse_scenario(l):
    global Scenario
    #l="cloud : Energy Consumed = 2421940.93607143"
    pattern = r'^Scenario (.*?)$'
    match = re.match(pattern, l)
    if match:
        Scenario=match.group(1)
        print(f"\n\n\n*******************************************\nScenario is {Scenario}\n*******************************************")





# For Tuple execution time
def parse_tuple_execution_time(l):
    global execution_time_df
    #l="MOTION_VIDEO_STREAM --->= 2.9571428571427987"
    pattern = r'^(.*?)\s+--->=\s+(\d+(\.\d+)?)$'
    match = re.match(pattern, l)
    if match:
        tuple_name=match.group(1)
        execution_time=float(match.group(2))
        data={'scenario':Scenario, 'tuple':tuple_name,'execution_time':execution_time}
        df=pd.DataFrame([data])
        #display(df)
        execution_time_df=pd.concat([execution_time_df,df],ignore_index=True)



# For loop delay
def parse_loops_delay(l):
    global loop_delay_df
    #l="MOTION_VIDEO_STREAM --->= 2.9571428571427987"
    #pattern = r'^(.*?) ([.*?])\s+--->=\s+(\d+(\.\d+)?)$'
    pattern = r'^(\S+)\s+\[(.*?)\]\s+--->=\s+(\d+\.\d+)$'

    match = re.match(pattern, l)
    if match:
        #app_name=match.group(1)
        loop_name=match.group(2)
        loop_name=f'[{loop_name}]'
        loop_delay=float(match.group(3))
        data={'scenario':Scenario, 'loop_name':loop_name,'delay':loop_delay}
        df=pd.DataFrame([data])
        #display(df)
        loop_delay_df=pd.concat([loop_delay_df,df],ignore_index=True)

        
#





# For total simulation time
def parse_simulation_time(l):
    #l="EXECUTION TIME = 370"
    pattern = r'^EXECUTION TIME = (\d+(\.\d+)?)$'
    global simulation_time_df
    match = re.match(pattern, l)
    if match:
        exec_time=match.group(1)
        data={'scenario':Scenario, 'simulation_time':exec_time}
        df=pd.DataFrame([data])
        #display(df)
        simulation_time_df=pd.concat([simulation_time_df,df],ignore_index=True)
        
        

        
def parse_networking_energy(l):
    global networking_energy_df
    pattern = r'^(.*?) : Total Networking Energy Consumption for this device = (\d+\.\d+) mJ$'
    match = re.match(pattern, l)
    if match:
        device = match.group(1)
        energy = float(match.group(2))
        data = {'scenario': Scenario,'device': device, 'nw_energy': energy}
        df=pd.DataFrame([data])
        #display(df)
        networking_energy_df = pd.concat([networking_energy_df, pd.DataFrame([data])], ignore_index=True)
        
        
        
def parse_computation_co2(l):
    global computation_co2_df

    # Pattern to match lines like "cloud : Application dcns total CO2 Emissions = 4.8294216965301E-5 kg"
    pattern = r'^(.*?): Application dcns total CO2 Emissions = (\d+\.\d+E[+-]?\d+) kg$'
    match = re.match(pattern, l)
    
    if match:
        device = match.group(1)
        emission = float(match.group(2))

        # Add parsed data to DataFrame
        data = {'scenario': Scenario, 'device': device, 'cmp_emission': emission}
        df=pd.DataFrame([data])
        #display(df)
        computation_co2_df = pd.concat([computation_co2_df, pd.DataFrame([data])], ignore_index=True)

                
                
                
                
def parse_networking_co2(l):
    global networking_co2_df
    pattern = r'^(.*?) : Total Networking CO2 Emissions for this device = (\d+\.\d+E[-+]?\d+) kg$'
    match = re.match(pattern, l)
    if match:
        device = match.group(1)
        co2 = float(match.group(2))
        data = {'scenario': Scenario, 'device': device, 'nw_emission': co2}
        df=pd.DataFrame([data])
        #display(df)
        networking_co2_df = pd.concat([networking_co2_df, pd.DataFrame([data])], ignore_index=True)



In [6]:
#line='dcns_0 [motion_detector, object_detector, object_tracker] --->= 1259.001302362737'
#parse_loops_delay(line)

In [7]:
# Parse
def parse(report_path=_report_path):
    with open(report_path, 'r') as file:
        for l in file:
            parse_scenario(l)
            parse_loops_delay(l)
            parse_simulation_time(l)
            parse_computation_energy(l)
            parse_networking_energy(l)
            parse_computation_co2(l)
            parse_networking_co2(l)
        

In [8]:
def save(save_path=_save_path):
    global data_frames
    
    # Combine all DataFrames into one merged DataFrame based on 'scenario' and 'device'
    '''merged_df = computation_energy_df.merge(computation_co2_df, on=['scenario', 'device'], how='outer') \
        .merge(networking_energy_df, on=['scenario', 'device'], how='outer') \
        .merge(networking_co2_df, on=['scenario', 'device'], how='outer') \
        .merge(loop_delay_df, on=['scenario'], how='outer') \
        .merge(simulation_time_df, on=['scenario'], how='outer')'''
    
    # Combine all DataFrames
    data_frames = {
        'Computation Energy Devices': computation_energy_df,
        'Networking Energy Devices': networking_energy_df,
        'Computation CO2 Emissions': computation_co2_df,
        'Networking CO2 Emissions': networking_co2_df,
        'Loop Delays': loop_delay_df,
        'Simulation Times': simulation_time_df
    }
     # Path to save the Excel file
    excel_path = os.path.join(save_path, 'Labelling_results.xlsx')
    csv_path = os.path.join(save_path, 'Labelling_results.csv')
    

    # Use Pandas Excel writer to save all DataFrames in one Excel file with multiple sheets
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for name, df in data_frames.items():
            if not df.empty:
                df.to_excel(writer, sheet_name=name, index=False)

    print(f"Data saved to {excel_path}")

In [9]:
def sort_scenarios():
    global data_frames

    #computation_energy_per_vm_df['energy']=computation_energy_per_vm_df['energy']/1000
    # Perform unit conversion from millijoules to joules for energy columns
    
    computation_energy_df['cmp_energy'] = computation_energy_df['cmp_energy'] / 1000  # Convert mj to Joules
    networking_energy_df['nw_energy'] = networking_energy_df['nw_energy'] / 1000  # Convert mj to Joules
    
    # Update column names to reflect new units
    #networking_energy_df.rename(columns={'nw_energy': 'nw_energy(J)'}, inplace=True)
    #computation_energy_df.rename(columns={'cmp_energy': 'cmp_energy(J)'}, inplace=True)
    #networking_co2_df.rename(columns={'nw_emission': 'nw_emission(kg)'}, inplace=True)
    #computation_co2_df.rename(columns={'cmp_emission': 'cmp_emission(kg)'}, inplace=True)
    
    data_frames = {
        'Computation Energy Devices': computation_energy_df,
        'Networking Energy Devices': networking_energy_df,
        'Computation CO2 Emissions': computation_co2_df,
        'Networking CO2 Emissions': networking_co2_df,
        'Loop Delays': loop_delay_df,
        'Simulation Times': simulation_time_df
    }
    for key in data_frames:
        df = data_frames[key]
        # Create a categorical Series for the 'scenario' column
        df['scenario'] = pd.Categorical(df['scenario'], categories=scenarios_list, ordered=True)
        
        # Sort the DataFrame based on the specified order
        df_sorted = df.sort_values(by=['scenario'])
        # Update the DataFrame in the dictionary
        data_frames[key] = df_sorted

In [10]:

run1=True
#scenarios_list=[ 'Edge_Based','Router_Only','Router_Proxy','Router_Cloud','Proxy_Only','Proxy_Cloud','Cloud_Only','Cloud_Based']
scenarios_list=['Cloud_Based']
if run1:
    _reset()
    for scenario in scenarios_list:
        run("LabellingDCNS", scenario)

In [11]:
# Parse the results
_reset()
parse()
sort_scenarios()
save()





*******************************************
Scenario is Cloud_Based
*******************************************
Data saved to Report_Saeedeh/Labelling_Results/Labelling_results.xlsx


In [12]:
computation_energy_df

,scenario,device,cmp_energy,app
0,Cloud_Based,cloud,31064.802943,dcns
1,Cloud_Based,proxy-server,0.000000,no_app
2,Cloud_Based,d-0,0.000000,no_app
3,Cloud_Based,m-0-0,0.000000,no_app
4,Cloud_Based,m-0-1,0.000000,no_app
5,Cloud_Based,m-0-2,0.000000,no_app
6,Cloud_Based,m-0-3,0.000000,no_app


In [13]:
import pandas as pd

# Load the CSV file (no need for full path if in the same directory)
df = pd.read_csv('Total_Labelling.csv')

# Print the first few rows to understand the structure
print(df.head())

                        Scenario  Edge_20msec  Router_only_20msec   
0   Cloud Computation Energy (j)        36.27               36.27  \
1   Proxy Computation Energy (j)         0.00                0.00   
2  Router Computation Energy (j)         0.00              480.05   
3     M_0 Computation Energy (j)       724.32              183.26   
4     M_1 Computation Energy (j)       724.32              183.26   

   Router_Proxy_20msec  Router_Cloud_20msec  Proxy_Only_20msec   
0                36.27               355.90              36.27  \
1               164.17                 0.00             485.99   
2               316.46               316.46               0.00   
3               183.26               183.26             183.26   
4               183.26               183.26             183.26   

   Proxy_Cloud_20msec  Cloud_Only_20msec  Cloud_Based_20msec  Edge_15msec   
0              349.70             892.08             1744.06        48.36  \
1              322.57             

In [14]:
import pandas as pd

# Load the CSV file again just in case we need a fresh start
df = pd.read_csv('Total_Labelling.csv')

# Set 'Scenario' as the index and transpose the DataFrame
df_cleaned = df.set_index('Scenario').T

# Rename the columns to represent the actual metrics (e.g., Cloud Energy, Router Energy)
df_cleaned.columns = df_cleaned.iloc[0]  # First row becomes column names
df_cleaned = df_cleaned.drop(df_cleaned.index[0])  # Drop the first row now that it's used

# Convert data to numeric (ignore errors for any non-numeric data)
df_cleaned = df_cleaned.apply(pd.to_numeric, errors='ignore')

# Reset index to make it easier to handle
df_cleaned.reset_index(inplace=True)

# Check the cleaned DataFrame
print(df_cleaned.head())


Edge_20msec                index   36.27     0.0     0.0  724.32  724.32   
0             Router_only_20msec   36.27    0.00  480.05  183.26  183.26  \
1            Router_Proxy_20msec   36.27  164.17  316.46  183.26  183.26   
2            Router_Cloud_20msec  355.90    0.00  316.46  183.26  183.26   
3              Proxy_Only_20msec   36.27  485.99    0.00  183.26  183.26   
4             Proxy_Cloud_20msec  349.70  322.57    0.00  183.26  183.26   

Edge_20msec  724.32  724.32   0.0   17.2  ...       0.0       0.0   7.4e-06   
0            183.26  183.26  0.00  17.20  ...  0.000007  0.000007  0.000007  \
1            183.26  183.26  0.00  18.06  ...  0.000007  0.000007  0.000007   
2            183.26  183.26  2.45  35.28  ...  0.000007  0.000007  0.000007   
3            183.26  183.26  0.00  18.06  ...  0.000007  0.000007  0.000007   
4            183.26  183.26  2.45  35.28  ...  0.000007  0.000007  0.000007   

Edge_20msec     4.3     1.1     5.4   2933.55   91.219  0.000387   1

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot CO2 Emissions or Energy for different devices and scenarios
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_cleaned, x='index', y='Cloud Computation CO2 (kg)', label='Cloud CO2')
sns.lineplot(data=df_cleaned, x='index', y='Proxy Computation CO2 (kg)', label='Proxy CO2')
sns.lineplot(data=df_cleaned, x='index', y='Router Computation CO2 (kg)', label='Router CO2')
plt.xlabel('Scenario')
plt.ylabel('CO2 Emissions (kg)')
plt.title('CO2 Emissions Across Scenarios')
plt.legend()
plt.show()


ValueError: Could not interpret value `Cloud Computation CO2 (kg)` for `y`. An entry with this name does not appear in `data`.

<Figure size 864x432 with 0 Axes>